# Самые близкие новости по семантике

In [1]:
import pandas as pd
import numpy as np

import pickle
import datetime

import gensim 
print(gensim.__version__)

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
matplotlib.style.use('ggplot')

%matplotlib inline

3.8.0


In [2]:
DATA_PATH = "../Сборщик новостей/news_data/rbc/rbc_news.pickle"

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)
    
# data =[item for item in data if item['date'][:4] == '2018'] + [data[913]]

# 1. Проставляем тэги

https://github.com/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb

In [ ]:
# https://rusvectores.org/ru/models/

In [9]:
import requests
from tqdm import tqdm
import re

from pymystem3 import Mystem
m = Mystem(entire_input=False)

# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mapping_url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split('\n'):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]


def tag_mystem(text='Текст нужно передать функции в виде строки!', mapping=None, postags=True):
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]
                else:
                    pos = 'X'
            tagged.append(lemma.lower() + '_' + pos)
        except:
            continue
    if not postags:
        tagged = [t.split('_')[0] for t in tagged]
    return tagged

# result = [ ]
for line in tqdm(data[274859:]):
    line["text_tagged"] = tag_mystem(text=re.sub(u"\ud83d\ude1d", "" ,line['text'].strip()), mapping=mystem2upos)
    line["title_tagged"] = tag_mystem(text=line["title"].strip(), mapping=mystem2upos)
    result.append(line)


 10%|█         | 2614/25094 [00:34<05:03, 74.16it/s]


 20%|██        | 5125/25094 [01:08<04:06, 80.97it/s]


 30%|███       | 7572/25094 [01:41<04:12, 69.50it/s]


 40%|███▉      | 10005/25094 [02:15<03:51, 65.10it/s]


 49%|████▉     | 12341/25094 [02:48<02:54, 73.27it/s]


...

 66%|██████▌   | 16605/25094 [04:32<02:13, 63.72it/s]


 76%|███████▌  | 18972/25094 [05:06<01:24, 72.31it/s]


 85%|████████▍ | 21295/25094 [05:39<00:49, 76.69it/s]


 94%|█████████▍| 23643/25094 [06:12<00:22, 63.26it/s]


100%|██████████| 25094/25094 [06:33<00:00, 63.70it/s]


'\n\nЗаместитель министра финансов России Антона Силуанова Алексей Саватюгин 18 января был отправлен в отставку. Информация об этом содержится в тексте распоряжения, опубликованного на сайте правительства. О том, что  чиновник покинет свой пост, стало известно еще в декабре 2012 г, но официальное подтверждение поступило только сейчас.\r\n\n\n\n\n\n\n\n                                                    Фото: РБК\n                                            \n\n\nА.Л.Саватюгин прокомментировал свое увольнение в микроблоге.  "Сегодня, первый раз в жизни я стал безработным. Несмотря на это и на московскую погоду, настроение замечательное! Психологи, это нормально?", - написал экс-чиновник.По словам бывшего заместителя министра финансов, в настоящее время он находится за рубежом. О погоде в столице экс-чиновник узнал из Интернета.А.Саватюгин занял высокий пост в системе Минфина при министре Алексее Кудрине в 2010 г. После отставки шефа чиновнику удалось сохранить свою должность.\r\n\n'

In [4]:
len(result)

274859

In [ ]:
with open("../Сборщик новостей/news_data/rbc/rbc_news_tag_lemm.pickle", 'wb') as f:
    pickle.dump(result, f)   

 92%|█████████▏| 274858/299953 [59:33<04:03, 103.17it/s]

In [ ]:
ban_words = """
вирус инфекция заразиться заболевание заражение умереть карантин заболеть заразить обострение заболеваемость
эпидемия пневмония болезнь инфекционный карантинный изоляция грипп осложнение госпитализация вирусный пандемии
эпидемии вируса коронавируса госпитализации карантинные карантина лекарств
"""

ban_words = set(tag_mystem(ban_words, mapping=mystem2upos))

# 2. TF-IDF dictionary

In [ ]:
tf_idf_texts = [' '.join(item['text_tagged'] + item['title_tagged']) for item in result]

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=10, max_df=0.8)
texts_v = vect.fit_transform(tf_idf_texts)

In [ ]:
feature_names = [' '.join(item.split('_')[:-1]) + '_' + item.split('_')[-1].upper() 
                         for item in vect.get_feature_names()]

tfidf_dict = dict(zip(feature_names, np.array(texts_v.mean(axis=0))[0]))

# 3. Text2Vec

In [ ]:
MODEL_PATH = "./model.bin"
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(MODEL_PATH, binary=True)

In [ ]:
cnt = 0
fuck_cnt = 0
D = w2v_model.vector_size

def get_vecor(text, tfidf_dict=tfidf_dict, w2v_model=w2v_model):
    global cnt
    global fuck_cnt
    D = w2v_model.vector_size
    
    vector = np.zeros(D)
    for word in text:
        cnt += 1
        if (word in tfidf_dict)&(word in w2v_model.vocab)&(word not in ban_words):
            vector = vector + tfidf_dict[word] * w2v_model.get_vector(word)
        else:
            fuck_cnt += 1
    return vector

In [ ]:
X = np.zeros((len(result), D))

for i,item in tqdm(enumerate(result)):
    text = item['text_tagged'] + item['title_tagged']
    vect = get_vecor(text)
    X[i] = vect

In [ ]:
fuck_cnt/cnt

In [ ]:
X.shape

# 4. Find liberalization texts

In [ ]:
# %%time
# from scipy.cluster import hierarchy
# from scipy.spatial.distance import pdist

# # pdist посчитает нам верхний треугольник матрицы попарных расстояний
# distance_mat = pdist(X, metric='cosine')

In [ ]:
rel_url = "https://www.rbc.ru/economics/30/03/2020/5e8230d89a7947e55652c618"
idx = [item['href'] for item in data].index(rel_url)
idx

In [ ]:
rel_vec = X[idx]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

X_similarity = cosine_similarity(X, np.array([rel_vec])).T[0]
similar = [(item['title'], jtem) for item,jtem in zip(data, X_similarity)]
similar = sorted(similar, key=lambda w: w[1], reverse=True)

In [ ]:
[x for x in similar[:200]]

In [ ]:
plt.figure(figsize=(10, 70))
Z = hierarchy.linkage(distance_mat, method='ward') # linkage — реализация агломеративного алгоритма
dend = hierarchy.dendrogram(Z, color_threshold=40, labels=names, leaf_font_size=12., orientation='left')

In [ ]:
%%time
import umap
embedding = umap.UMAP(n_neighbors=5).fit_transform(X)

In [ ]:
plt.figure(figsize=(12,12))

points = embedding
plt.scatter(points[:,0], points[:,1]);

In [ ]:
metric='cosine'